In [21]:
# output 디렉토리 초기화 - 없으면 생성, 있으면 내부 jpg 파일 삭제

import os

if os.path.exists("./output"):
    listdir = os.listdir("./output")
    for f in listdir:
        if f.endswith(".jpg"):
            os.remove(f"./output/{f}")
else:
    os.makedirs("./output")

if os.path.exists("./output.zip"):
    os.remove("./output.zip")

In [ ]:
# 카메라 입력 받아온 후 녹화

try:
    import jchm
except ModuleNotFoundError:
    pass
import cv2
import os
import time
from contextlib import suppress
import numpy
from jchutils.camera import Camera, CameraMode

def main():
    OUTPUT_DIR = "./output"
    FPS = 10
    DT = 1.0 / FPS
    CAMERA = "lidar"
    JPG_COMPRESS_QUALITY = 80
    REALTIME_SHOW_QUALITY = 80

    camera = Camera(
        mode=CameraMode.JAJUCHA,
        device=CAMERA
    )

    with suppress(KeyboardInterrupt):
        cnt = 0
        next_frame = time.perf_counter()
        while True:
            next_frame += DT
            now = time.perf_counter()
            if now < next_frame:
                time.sleep(next_frame - now)
            else:
                missed = int((now - next_frame) // DT) + 1
                next_frame += missed * DT
            cnt += 1
            img = camera.getFrame()
            camera.showFrame(img, REALTIME_SHOW_QUALITY)
            camera.saveFrame(img, OUTPUT_DIR, f"{cnt}.jpg", JPG_COMPRESS_QUALITY)

if __name__ == "__main__":
    main()

In [ ]:
# output 디렉토리 압축

import shutil

!cd output && find . -exec touch -t 200001010000 {} \;

shutil.make_archive("./output", "zip", "./output/")

In [ ]:
# 파일 HTTP 정적 서빙 + 업로드

from jchutils.fileserver import FileServer

fileserver = FileServer("./", "0.0.0.0", 8000)
fileserver.runForever()